# Kata Kunci Berita

Mencari dokumen melalui kata kunci dapat mempermudah kita dalam mencari suatu dokumen yang relevan. Kata Kunci merupakan kata yang penting dan sering muncul dalam sebuah teks maupun dokumen. Biasanya kata kunci mengandung kata yang unik, dan bermakna spesifik.


## Proses Pembersihan Teks


### Import Library


In [1]:
# Library untuk text preprocessing
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

# Library untuk data manipulation
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm

# Library untuk text similarity
from sklearn.metrics.pairwise import cosine_similarity 

### Load Data


In [2]:
data = pd.read_csv('data-artikel-cnbc.csv')
data.head()

,judul,tanggal,isi,url,kategori
0,Bukan Kaleng-kaleng! Nilai Skuad Timnas Indone...,09 September 2024 19:30,"Jakarta, CNBC Indonesia -Kedatangan pemain-pem...",https://www.cnbcindonesia.com/research/2024090...,Research
1,"Aset Bank Mandiri Tembus Rp 2.200 T, Paling Ju...",09 September 2024 19:05,"Jakarta, CNBC Indonesia -Sepanjang paruh perta...",https://www.cnbcindonesia.com/research/2024090...,Research
2,Emas Meredup Saat Suku Bunga The Fed Mau Turun...,09 September 2024 18:10,"Jakarta,CNBC Indonesia -Harga emas dunia melem...",https://www.cnbcindonesia.com/research/2024090...,Research
3,"Tetangga RI Dihantam 'Bencana' Iklim, Harga Ba...",09 September 2024 15:55,"Jakarta, CNBC Indonesia -Indeks Harga Konsumen...",https://www.cnbcindonesia.com/research/2024090...,Research
4,"Tantama Hingga Perwira, Ini Urutan Lengkap Pan...",09 September 2024 14:35,"Jakarta, CNBC Indonesia-Tentara Nasional Indon...",https://www.cnbcindonesia.com/research/2024090...,Research


In [3]:
# Filter data untuk kategori "Research" dan "News" masing-masing diambil 50 berita
research = data[data["kategori"] == "Research"].sample(n=50, random_state=42)
news = data[data["kategori"] == "News"].sample(n=50, random_state=42)

In [4]:
# Gabungkan keduanya
data = pd.concat([research, news], ignore_index=True)

### Preprocessing Text


In [5]:
def clean_text(text):
	text = re.sub(r'((www\.[^\s]+)|(https?://[^\s]+))', ' ', text) # Menghapus https* and www*
	text = re.sub(r'@[^\s]+', ' ', text) # Menghapus username
	text = re.sub(r'[\s]+', ' ', text) # Menghapus tambahan spasi
	text = re.sub(r'#([^\s]+)', ' ', text) # Menghapus hashtags
	text = re.sub(r"[^a-zA-Z : .]", "", text) # Menghapus tanda baca
	text = re.sub(r'\d', ' ', text) # Menghapus angka
	text = text.lower()
	text = text.encode('ascii','ignore').decode('utf-8') #Menghapus ASCII dan unicode
	text = re.sub(r'[^\x00-\x7f]',r'', text)
	text = text.replace('\n','') #Menghapus baris baru
	text = text.strip()
	return text

def clean_stopword(tokens):
	listStopword =  set(stopwords.words('indonesian'))
	filtered_words = [word for word in tokens if word.lower() not in listStopword]
	return filtered_words

In [6]:
def preprocess_text(content):
	result = {}
	for i, text in enumerate(tqdm(content)):
		cleaned_text = clean_text(text)
		tokens = word_tokenize(cleaned_text)
		cleaned_stopword = clean_stopword(tokens)
		result[i] = ' '.join(cleaned_stopword)
	return result

data['cleaned_news'] = preprocess_text(data['isi'])
data

100%|███████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 222.34it/s]


,judul,tanggal,isi,url,kategori,cleaned_news
0,Agenda Hari Ini: Paus Fransiskus Bertemu Jokow...,04 September 2024 06:15,"Jakarta, CNBC Indonesia -Sejumlah agenda dan e...",https://www.cnbcindonesia.com/research/2024090...,Research,jakarta cnbc indonesia agenda event digelar ra...
1,11 Provinsi Ini Alami Deflasi Parah Beruntun: ...,05 September 2024 13:50,"Jakarta, CNBC Indonesia-Memasuki September 202...",https://www.cnbcindonesia.com/research/2024090...,Research,jakarta cnbc indonesiamemasuki september indon...
2,Peringatan Keras! Ini 13 Wilayah yang Berpoten...,04 September 2024 16:05,"Jakarta, CNBC Indonesia- Gempamegathrusthingga...",https://www.cnbcindonesia.com/research/2024090...,Research,jakarta cnbc indonesia gempamegathrusthingga t...
3,"""Emas Hijau"", Harta Karun Terpendam dari Belan...",05 September 2024 18:30,"Jakarta,CNBC Indonesia-Gaharu, juga dikenal se...",https://www.cnbcindonesia.com/research/2024090...,Research,jakartacnbc indonesiagaharu dikenal agarwood j...
4,Hierarki Gereja Katolik Roma dari Paus Hingga ...,05 September 2024 20:25,"Jakarta, CNBC Indonesia -Dalam ajaran resmi Ge...",https://www.cnbcindonesia.com/research/2024090...,Research,jakarta cnbc indonesia ajaran resmi gereja kat...
...,...,...,...,...,...,...
95,"Kinerja Moncer, PIS Terbukti Jadi Ujung Tombak...",09 September 2024 17:25,"Jakarta, CNBC Indonesia -PT Pertamina Internat...",https://www.cnbcindonesia.com/news/20240909151...,News,jakarta cnbc indonesia pt pertamina internatio...
96,"Kuota BBM Subsidi di 2025 Lebih Kecil, Ternyat...",09 September 2024 09:49,"Jakarta, CNBC Indonesia -Badan Pengatur Hilir ...",https://www.cnbcindonesia.com/news/20240909094...,News,jakarta cnbc indonesia badan pengatur hilir mi...
97,"Pabrik Kompor Quantum PHK 511 Karyawan, Utangn...",09 September 2024 16:53,"Jakarta, CNBC Indonesia- Ratusan pekerja yang ...",https://www.cnbcindonesia.com/news/20240909164...,News,jakarta cnbc indonesia ratusan pekerja tergabu...
98,"Sri Mulyani Suntik Satgas BLBI Rp10,2 M Demi K...",09 September 2024 12:55,"Jakarta, CNBC Indonesia- Kementerian Keuangan ...",https://www.cnbcindonesia.com/news/20240909121...,News,jakarta cnbc indonesia kementerian keuangan ke...


### Memisahkan Kalimat menjadi per Kata


In [7]:
kalimat = nltk.sent_tokenize(data['cleaned_news'][4])
kalimat = [sentence.replace(' .', '') for sentence in kalimat]
kata = nltk.word_tokenize(data['cleaned_news'][4])
kata = list(set(k for k in kata if k != '.'))

In [8]:
kalimat

['jakarta cnbc indonesia ajaran resmi gereja katolik struktur hierarkis terpisahkan kehidupan misinya',
 'kristus mengutus rasul menyebarkan injil misi zaman',
 'konsili vatikan ii uskup menggantikan rasul gembala gereja',
 'gereja dikenal berkembang kelompok pengikut yesus gereja rasul',
 'struktur terbentuk periode kebangkitan yesus kemartiran st. ignatius antiokhia abad',
 'hierarki gereja katolik paus pemimpin tertinggi uskup imam diakon',
 'paus meneruskan santo petrus pemimpin uskup uskup roma memiliki peran pemimpin gereja universal',
 'dilansir imankatolik.or.id tradisi katolik roma dipandang pusat gereja paus dipilih kardinal penasihat paus',
 'uskup menggantikan rasul berperan pemimpin umat wilayah masingmasing bersamasama membentuk dewan uskup',
 'peran uskup memimpin komunitas lokal',
 'pemimpin spiritual bertanggung mempersatukan umat tugas utama : pewartaan injil perayaan sakramen pelayanan umat',
 'mendukung tugas uskup dibantu imam diakon',
 'imam berperan perpanjangan 

## Pembuatan Dataframe

### Persiapan Kalimat dan Kata

In [9]:
vocabulary = set()
for sentence in kalimat:
  for word in sentence.split():
    vocabulary.add(word)

print(kalimat)
print(vocabulary)

['jakarta cnbc indonesia ajaran resmi gereja katolik struktur hierarkis terpisahkan kehidupan misinya', 'kristus mengutus rasul menyebarkan injil misi zaman', 'konsili vatikan ii uskup menggantikan rasul gembala gereja', 'gereja dikenal berkembang kelompok pengikut yesus gereja rasul', 'struktur terbentuk periode kebangkitan yesus kemartiran st. ignatius antiokhia abad', 'hierarki gereja katolik paus pemimpin tertinggi uskup imam diakon', 'paus meneruskan santo petrus pemimpin uskup uskup roma memiliki peran pemimpin gereja universal', 'dilansir imankatolik.or.id tradisi katolik roma dipandang pusat gereja paus dipilih kardinal penasihat paus', 'uskup menggantikan rasul berperan pemimpin umat wilayah masingmasing bersamasama membentuk dewan uskup', 'peran uskup memimpin komunitas lokal', 'pemimpin spiritual bertanggung mempersatukan umat tugas utama : pewartaan injil perayaan sakramen pelayanan umat', 'mendukung tugas uskup dibantu imam diakon', 'imam berperan perpanjangan tangan uskup

### Implementasi ke Dataframe

In [10]:
# Membuat Dataframe kosong dengan kolom sesuai dengan vocabulary
pd.set_option('future.no_silent_downcasting', True)
df = pd.DataFrame(columns=list(vocabulary), index=kalimat)

# Mengisi nilai kosong dengan 0
df = df.fillna(0)  # Fill with 0s

# Menghitung nilai setiap kata pada kalimat
for i, sentence in enumerate(kalimat):
  for word in sentence.split():
    df.loc[sentence, word] += 1

# Melihat isi Dataframe
df

,misi,jakarta,berkesinambungan,mengutus,ii,pengambilan,hardjoatmodjo,abad,menggembalakan,kegiatan,...,batu,khusus,legitimasi,diakon,masingmasing,berkembang,diangkat,hierarki,roma,mengikat
jakarta cnbc indonesia ajaran resmi gereja katolik struktur hierarkis terpisahkan kehidupan misinya,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
kristus mengutus rasul menyebarkan injil misi zaman,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
konsili vatikan ii uskup menggantikan rasul gembala gereja,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
gereja dikenal berkembang kelompok pengikut yesus gereja rasul,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
struktur terbentuk periode kebangkitan yesus kemartiran st. ignatius antiokhia abad,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
hierarki gereja katolik paus pemimpin tertinggi uskup imam diakon,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
paus meneruskan santo petrus pemimpin uskup uskup roma memiliki peran pemimpin gereja universal,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
dilansir imankatolik.or.id tradisi katolik roma dipandang pusat gereja paus dipilih kardinal penasihat paus,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
uskup menggantikan rasul berperan pemimpin umat wilayah masingmasing bersamasama membentuk dewan uskup,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
peran uskup memimpin komunitas lokal,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Keyword Extraction

In [11]:
# Menghitung jumlah kemunculan pada tiap kata
word_frequencies = df.sum(axis=0)

# Mengurutkan berdasarkan jumlah kemunculan kata
sorted_word_frequencies = word_frequencies.sort_values(ascending=False)

# Menampilkan kata teratas
rank = 5

for i, (word, freq) in enumerate(sorted_word_frequencies.items()):
    if i < rank:
        print(f"Rank {i+1}: {word} (Frequency: {freq})")

Rank 1: gereja (Frequency: 15)
Rank 2: uskup (Frequency: 12)
Rank 3: paus (Frequency: 11)
Rank 4: katolik (Frequency: 7)
Rank 5: kardinal (Frequency: 7)
